# movie_mixer

Recomendation system to Recomend movies based on combined attributes of two or more movies.

[movielense-small dataset](https://www.kaggle.com/movielens/movielens-20m-dataset) is used for this project.

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns


movies = pd.read_csv("")